In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\mariu\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
import pandas as pd
import logging
import re
from difflib import get_close_matches

# Enable debugging
DEBUG = True

def debug_print(message):
    if DEBUG:
        print(message)

# Configure logging
logging.basicConfig(filename='rename_log.txt', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Load CSV data
csv_path = 'Max_Turkey_sample.csv'  # Updated path
debug_print(f"CSV Path: {csv_path}")

try:
    df = pd.read_csv(csv_path)
    debug_print("CSV file loaded successfully.")
except Exception as e:
    debug_print(f"Error loading CSV: {e}")
    raise

data = {}
matched_titles = set()

def normalize(text):
    if isinstance(text, str):
        return text.strip().lower()
    return ''

def find_best_match(folder_name, data_keys):
    matches = get_close_matches(folder_name, data_keys, n=1, cutoff=0.6)
    return matches[0] if matches else None

def is_valid_scrid(scrid):
    return scrid.isdigit()

debug_print("Processing CSV data...")

for index, row in df.iterrows():
    program_title = normalize(row['A | Program Title'])
    debug_print(f"Normalized Program Title from CSV: {program_title}")
    if program_title:
        if program_title not in data:
            data[program_title] = []

        data[program_title].append({
            'show_scrid': str(row.get('A - CFPS | Show Scrid #', '')).strip(),
            'episode_scrid': str(row.get('A - CFPS | Episode Scrid #', '')).strip(),
            'season_number': str(row.get('A | Season', '')).strip(),
            'episode_number': str(row.get('A | Episode Number', '')).strip()
        })

debug_print(f"Loaded data for {len(data)} programs.")
print(data)


CSV Path: Max_Turkey_sample.csv
CSV file loaded successfully.
Processing CSV data...
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 7 faces
Normalized Program Title from CSV: 140journos - sikismislik
Normalized Program Title from CSV: 140journos - sikismislik
Normalized Program Title from CSV: 140journos - sikismislik
Normalized Program Title from CSV: 140journos - sikismislik
Normalized Program Title from CSV: 140journos - sikismislik
Normalized Program Title from CSV: 140journos - sikismislik
Normalized Program Title from CSV: agency
Normalized Program Title from CSV: agency
Normalized Program Title from CSV: agency

In [6]:
turkey_folder = 'Turkey'  # Updated path
debug_print(f"Turkey Folder Path: {turkey_folder}")

if not os.path.exists(turkey_folder):
    debug_print("Error: Turkey folder path does not exist.")
    raise FileNotFoundError(f"Turkey folder path does not exist: {turkey_folder}")

for dirpath, dirnames, filenames in os.walk(turkey_folder):
    debug_print(f"Walking directory: {dirpath}")

    current_folder = normalize(os.path.basename(dirpath))
    debug_print(f"Current folder for matching: {current_folder}")

    program_key = find_best_match(current_folder, data.keys())
    debug_print(f"Best match found: {program_key}")
    program_data = data.get(program_key)

    if program_data:
        matched_titles.add(program_key)

    if not program_data:
        debug_print(f"No matching program title for folder: {current_folder}")
        continue  # Skip folders without matching program title

    for root, _, files in os.walk(dirpath):
        for file in files:
            if file.lower().endswith(('.jpg', '.png')):
                old_file_path = os.path.join(root, file)
                debug_print(f"Processing file: {old_file_path}")

                # Skip if SCRID already exists and matches
                if re.search(r'SCRID-(\d+)', file):
                    existing_scrid = re.search(r'SCRID-(\d+)', file).group(1)
                    if existing_scrid == program_data[0]['show_scrid']:
                        debug_print(f"Skipped (SCRID exists): {file}")
                        continue

                if 'artwork' in root.lower():
                    show_scrid = program_data[0]['show_scrid']
                    if is_valid_scrid(show_scrid):
                        # Remove existing prefix up to the first underscore and add new SCRID
                        new_name = re.sub(r'^.*?_', f'SCRID-{show_scrid}_', file)
                        debug_print(f"Artwork - New name: {new_name}")
                    else:
                        logging.warning(f"Invalid SCRID for {file}. Skipping.")
                        debug_print(f"Invalid SCRID for {file}. Skipping.")
                        continue

                elif 'episodes' in root.lower():
                    # Extract season number from folder path
                    season_match = re.search(r's(\d+)', root, re.IGNORECASE)
                    season_number = season_match.group(1) if season_match else ''

                    # Extract episode number from filename
                    episode_match = re.match(r'(\d+)_s(\d+)_', file, re.IGNORECASE)
                    episode_number = episode_match.group(1) if episode_match else ''

                    episode_data = next(
                        (ep for ep in program_data if ep['season_number'] == season_number and ep['episode_number'] == episode_number),
                        None
                    )

                    if episode_data and is_valid_scrid(episode_data['episode_scrid']):
                        new_name = f"SCRID-{episode_data['episode_scrid']}_{file}"
                        debug_print(f"Episode - New name: {new_name}")
                    else:
                        logging.warning(f"No valid SCRID for episode {file}. Skipping.")
                        debug_print(f"No valid SCRID for episode {file}. Skipping.")
                        continue
                else:
                    debug_print(f"Skipping unrelated folder: {root}")
                    continue

                new_file_path = os.path.join(root, new_name)

                if os.path.exists(new_file_path):
                    logging.warning(f"Skipped: {new_file_path} already exists.")
                    debug_print(f"Skipped (exists): {new_file_path}")
                    continue

                try:
                    os.rename(old_file_path, new_file_path)
                    debug_print(f"Renamed: {old_file_path} to {new_file_path}")
                except Exception as e:
                    logging.error(f"Error renaming {old_file_path}: {e}")
                    debug_print(f"Error renaming {old_file_path}: {e}")

# Log missing titles
missing_titles = set(data.keys()) - matched_titles
for title in missing_titles:
    logging.warning(f"Missing Title in Files: {title}")

print("Renaming process completed. Check rename_log.txt for details.")

Turkey Folder Path: Turkey
Walking directory: Turkey
Current folder for matching: turkey
Best match found: None
No matching program title for folder: turkey
Walking directory: Turkey\00
Current folder for matching: 00
Best match found: None
No matching program title for folder: 00
Walking directory: Turkey\00\140journos - Sikismislik
Current folder for matching: 140journos - sikismislik
Best match found: 140journos - sikismislik
Processing file: Turkey\00\140journos - Sikismislik\Artwork\EN\SCRID-12345678_[140journosSikismisklik_S1_1680x3636]_LAN-zxx_PUR-tile.jpg
Skipped (SCRID exists): SCRID-12345678_[140journosSikismisklik_S1_1680x3636]_LAN-zxx_PUR-tile.jpg
Processing file: Turkey\00\140journos - Sikismislik\Artwork\EN\SCRID-12345678_[140journosSikismisklik_S1_1920x1080]_LAN-en-US_PUR-logo.png
Skipped (SCRID exists): SCRID-12345678_[140journosSikismisklik_S1_1920x1080]_LAN-en-US_PUR-logo.png
Processing file: Turkey\00\140journos - Sikismislik\Artwork\EN\SCRID-12345678_[140journosSiki

WORKING VERSION for ALL